In [1]:
import pandas as pd
import numpy as np

import glob

print('>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!')
f'pd : {pd.__version__}  |  np : {np.__version__}'

>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!


'pd : 1.0.5  |  np : 1.16.5'

## 개별 테이블 로드 및 내용 확인

In [2]:
time_df = pd.read_parquet('../ETL_source_output/제주도_월별_행정동별_시간대별_총합.parquet')
time_df.columns = ['시군구코드', '시군구명', '행정동코드', '행정동명', '연도별', '월별', '시간대별', '유동인구총합']
time_df

,시군구코드,시군구명,행정동코드,행정동명,연도별,월별,시간대별,유동인구총합
0,50110,제주시,50110250,한림읍,2019,1,0,33086
1,50110,제주시,50110250,한림읍,2019,1,1,25067
2,50110,제주시,50110250,한림읍,2019,1,2,20057
3,50110,제주시,50110250,한림읍,2019,1,3,18365
4,50110,제주시,50110250,한림읍,2019,1,4,17390
...,...,...,...,...,...,...,...,...
16507,50130,서귀포시,50130610,중문동,2020,4,19,152315
16508,50130,서귀포시,50130610,중문동,2020,4,20,114996
16509,50130,서귀포시,50130610,중문동,2020,4,21,86585
16510,50130,서귀포시,50130610,중문동,2020,4,22,59048


In [3]:
sex_age_df = pd.read_parquet('../ETL_source_output/제주도_월별_행정동별_성연령별_총합.parquet')
sex_age_df['sex'] = sex_age_df.sex_age.str[0].str.upper()
sex_age_df['age'] = sex_age_df.sex_age.str[1:].astype(int)
sex_age_df = sex_age_df.drop(columns=['sex_age'])[['sgg_cd', 'sgg_nm', 'admdong_cd', 'admdong_nm',
                                                   'year', 'month', 'sex', 'age', 'total']]
sex_age_df.columns = ['시군구코드', '시군구명', '행정동코드', '행정동명', '연도별', '월별', '성별', '연령대별', '유동인구총합']
sex_age_df

,시군구코드,시군구명,행정동코드,행정동명,연도별,월별,성별,연령대별,유동인구총합
0,50110,제주시,50110250,한림읍,2019,1,M,0,414
1,50110,제주시,50110250,한림읍,2019,1,M,10,13659
2,50110,제주시,50110250,한림읍,2019,1,M,15,93933
3,50110,제주시,50110250,한림읍,2019,1,M,20,141344
4,50110,제주시,50110250,한림읍,2019,1,M,25,214588
...,...,...,...,...,...,...,...,...,...
19259,50130,서귀포시,50130610,중문동,2020,4,F,50,150554
19260,50130,서귀포시,50130610,중문동,2020,4,F,55,124412
19261,50130,서귀포시,50130610,중문동,2020,4,F,60,69596
19262,50130,서귀포시,50130610,중문동,2020,4,F,65,21685


## 월별 행정동별-자치구별-시도전체 테이블 생성

In [4]:
dong_df = time_df.groupby(['시군구코드', '시군구명', '행정동코드', '행정동명', '연도별', '월별'])['유동인구총합'].sum().reset_index()
dong_df

,시군구코드,시군구명,행정동코드,행정동명,연도별,월별,유동인구총합
0,50110,제주시,50110250,한림읍,2019,1,5166548
1,50110,제주시,50110250,한림읍,2019,2,4869841
2,50110,제주시,50110250,한림읍,2019,3,6316911
3,50110,제주시,50110250,한림읍,2019,4,6687991
4,50110,제주시,50110250,한림읍,2019,5,6990796
...,...,...,...,...,...,...,...
683,50130,서귀포시,50130620,예래동,2019,12,3752349
684,50130,서귀포시,50130620,예래동,2020,1,3584819
685,50130,서귀포시,50130620,예래동,2020,2,1730591
686,50130,서귀포시,50130620,예래동,2020,3,1858855


In [5]:
gu_df = dong_df.groupby(['시군구코드', '시군구명', '연도별', '월별'])['유동인구총합'].sum().reset_index()

print(gu_df.shape)
gu_df.head()

(32, 5)


,시군구코드,시군구명,연도별,월별,유동인구총합
0,50110,제주시,2019,1,266221785
1,50110,제주시,2019,2,242916949
2,50110,제주시,2019,3,288079349
3,50110,제주시,2019,4,282092161
4,50110,제주시,2019,5,292551360


In [6]:
sido_df = gu_df.groupby(['연도별', '월별'])['유동인구총합'].sum().reset_index()

print(sido_df.shape)
sido_df.head()

(16, 3)


,연도별,월별,유동인구총합
0,2019,1,327452745
1,2019,2,296060754
2,2019,3,348623883
3,2019,4,341913102
4,2019,5,354824438


## 최종 엑셀 파일 저장 및 확인

In [7]:
with pd.ExcelWriter('../ETL_source_output/제주도_유동인구_집계_최종.xlsx') as writer:
    sido_df.to_excel(writer, sheet_name='월별 시도별 유동인구', index=False)
    gu_df.to_excel(writer, sheet_name='월별 시군구별 유동인구', index=False)
    dong_df.to_excel(writer, sheet_name='월별 행정동별 유동인구', index=False)
    sex_age_df.to_excel(writer, sheet_name='월별 성연령별 유동인구', index=False)
    time_df.to_excel(writer, sheet_name='월별 시간대별 유동인구', index=False)

# 파일 저장 확인
glob.glob('../ETL_source_output/제주도_유동인구_집계_최종.xlsx')

['../ETL_source_output/제주도_유동인구_집계_최종.xlsx']